In [1]:
from datetime import datetime
import requests
import pandas as pd
import pickle
from flask import Flask, request, render_template

In [2]:
# Load the trained ARIMA model from the file
with open('model.pkl', 'rb') as handle:
    model = pickle.load(handle)

In [3]:
# Build the Flask API endpoint
app = Flask(__name__)


@app.route('/', methods=['GET', 'POST'])
def home():
    if request.method == 'POST':
        # Get the user input from the HTML form
        location = request.form['location']
        date_str = request.form['date'] + ' ' + request.form['time']

        # Convert the user input to a pandas datetime object
        date = pd.to_datetime(date_str)

        # Retrieve the weather data for the given date/time and location
        weather_data = get_weather_data(location, date)

        # Preprocess the weather data
        weather_data = preprocess_weather_data(weather_data)

        # Use the trained ARIMA model to predict the bike availability
        prediction = model.forecast(steps=1, exog=[weather_data])[0]

        # Return the predicted bike availability to the user on the web page
        return render_template('index.html', prediction=prediction)

    else:
        # Render the HTML form for the user to input the bike station location and date/time
        return render_template('index.html')


In [4]:
def get_weather_data(location, date):
    # Call an external API like OpenWeatherMap to retrieve the weather data for the given location and date/time
    # You will need an API key from OpenWeatherMap to use this API
    api_key = 'YOUR_API_KEY'
    base_url = 'https://api.openweathermap.org/data/2.5/weather'
    params = {'q': location, 'appid': api_key}
    response = requests.get(base_url, params=params)
    weather_data = response.json()

    # Extract the relevant weather data and return it as a dictionary
    extracted_data = {'temperature': weather_data['main']['temp'],
                      'wind_speed': weather_data['wind']['speed'],
                      'pressure': weather_data['main']['pressure'],
                      'humidity': weather_data['main']['humidity']}
    return extracted_data

In [5]:
def preprocess_weather_data(weather_data):
    # Preprocess the weather data to transform it to the same format as the training data
    # For example, you can apply the same scaling or normalization as used during training
    # You can also engineer additional features if required
    # Finally, return the preprocessed weather data as a pandas dataframe
    preprocessed_data = pd.DataFrame([weather_data])
    preprocessed_data['temperature'] = preprocessed_data['temperature'] - 273.15
    return preprocessed_data


if __name__ == '__main__':
    app.run(debug=True)

 * Serving Flask app '__main__'
 * Debug mode: on


 * Running on http://127.0.0.1:5000
Press CTRL+C to quit
 * Restarting with stat


SystemExit: 1

c:\Users\Olga\anaconda3\envs\comp30830\lib\site-packages\IPython\core\interactiveshell.py:3513: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)
